# TP3 - Exercise 3
## Mateus Ribeiro da Silva

In [46]:
#Importing some libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from scipy.stats import norm
from sklearn.utils import shuffle
from sklearn import neighbors
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler


In [23]:
#Import dataset
attributes_spotify = pd.read_csv("https://raw.githubusercontent.com/emmanueliarussi/DataScienceCapstone/master/3_MidtermProjects/ProjectBOM/data/attributes_spotify.csv")
attributes_spotify = attributes_spotify.sample(frac=1).reset_index(drop=True)
attributes_spotify.describe()

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target
count,2017.000000,2017.000000,2017.000000,2.017000e+03,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000
mean,1008.000000,0.187590,0.618422,2.463062e+05,0.681577,0.133286,5.342588,0.190844,-7.085624,0.612295,0.092664,121.603272,3.968270,0.496815,0.505702
std,582.402066,0.259989,0.161029,8.198181e+04,0.210273,0.273162,3.648240,0.155453,3.761684,0.487347,0.089931,26.685604,0.255853,0.247195,0.500091
min,0.000000,0.000003,0.122000,1.604200e+04,0.014800,0.000000,0.000000,0.018800,-33.097000,0.000000,0.023100,47.859000,1.000000,0.034800,0.000000
25%,504.000000,0.009630,0.514000,2.000150e+05,0.563000,0.000000,2.000000,0.092300,-8.394000,0.000000,0.037500,100.189000,4.000000,0.295000,0.000000
50%,1008.000000,0.063300,0.631000,2.292610e+05,0.715000,0.000076,6.000000,0.127000,-6.248000,1.000000,0.054900,121.427000,4.000000,0.492000,1.000000
75%,1512.000000,0.265000,0.738000,2.703330e+05,0.846000,0.054000,9.000000,0.247000,-4.746000,1.000000,0.108000,137.849000,4.000000,0.691000,1.000000
max,2016.000000,0.995000,0.984000,1.004627e+06,0.998000,0.976000,11.000000,0.969000,-0.307000,1.000000,0.816000,219.331000,5.000000,0.992000,1.000000


In [24]:
pd.set_option('display.max_columns', None)
attributes_spotify.head()

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,1436,0.001010,0.466,16588,0.943,0.000000,6,0.4580,-3.545,0,0.3850,113.457,5.0,0.5980,0,dance alot,Various Artists
1,745,0.351000,0.561,155040,0.329,0.031100,2,0.1160,-12.680,1,0.0282,105.029,4.0,0.2970,1,White Rabbit,Jefferson Airplane
2,1333,0.000111,0.450,267000,0.986,0.251000,1,0.0645,-2.384,1,0.1510,159.976,4.0,0.1720,0,Ways Of Transfiguration,Buried Side
3,723,0.591000,0.624,251627,0.724,0.000008,10,0.1010,-7.113,1,0.1010,78.008,4.0,0.6200,1,Step,Vampire Weekend
4,487,0.019300,0.352,248133,0.807,0.000006,0,0.0963,-7.781,1,0.1730,90.772,4.0,0.0459,1,Don't Die,Killer Mike


In [93]:
attributes_spotify.corr()['target']

Unnamed: 0         -0.865969
acousticness       -0.129627
danceability        0.176706
duration_ms         0.146749
energy              0.039688
instrumentalness    0.152594
key                 0.033594
liveness            0.026364
loudness           -0.072000
mode               -0.072336
speechiness         0.154006
tempo               0.034732
time_signature      0.040182
valence             0.107930
target              1.000000
song_title         -0.000058
artist             -0.014588
Name: target, dtype: float64

### Taking a look in dataset we can see some variables that don't give us much information about target, for example, the song title, in most of the cases, didn't define if someone will like it. Therefore, variables like 'song_title', 'artist', 'key', 'mode' and 'time_signature' cannot be useful. However, let's consider in first case all the variables and see how is the classifier's behaviour.

In [94]:
#Transform some attributes ('song title' and 'artist') into dummy variables in order to enable the classifier compute.
le = LabelEncoder()
attributes_spotify['song_title'] = le.fit_transform(attributes_spotify['song_title'])
attributes_spotify['artist'] = le.fit_transform(attributes_spotify['artist'])

In [99]:
#Separate varibles from targets
variables_columns = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', 'song_title', 'artist']
X, y = attributes_spotify[variables_columns], attributes_spotify['target']

In [100]:
#Instance KNN Classifier
knn = neighbors.KNeighborsClassifier()

#Create a hyperparameters dictionary for GridSeach using CrossValidation
params_grid = {'n_neighbors': [k for k in range(1, 100)]}

#Instance GridSearchCV to tesl all k's using 10 cross validations split.
knn_grid = GridSearchCV(knn, params_grid, cv=10)

#Fit model
knn_grid.fit(X,y)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30, ...]})

In [101]:
#Find the best performance
knn_grid.best_params_

{'n_neighbors': 37}

In [102]:
#Find the best score performance
knn_grid.best_score_

0.5909979803950545

### Using all variables, except 'Unnamed' column, we got 59.1% of accuracy, which is a bad result. Discarding some variables previously mentioned...

In [107]:
#Separate varibles from targets
variables_columns = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness','speechiness', 'tempo','valence']
X, y = attributes_spotify[variables_columns], attributes_spotify['target']

In [108]:
#Instance KNN Classifier
knn = neighbors.KNeighborsClassifier()

#Create a hyperparameters dictionary for GridSeach using CrossValidation
params_grid = {'n_neighbors': [k for k in range(1, 100)]}

#Instance GridSearchCV to tesl all k's using 10 cross validations split.
knn_grid = GridSearchCV(knn, params_grid, cv=10)

#Fit model
knn_grid.fit(X,y)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30, ...]})

In [109]:
#Find the best performance
knn_grid.best_params_

{'n_neighbors': 39}

In [110]:
#Find the best score performance
knn_grid.best_score_

0.599923649081326

### Choosing only the possible relevant variables we got 59.99% of accuracy. Although the result is better than the last one, such perfomance is bad yet. We are measuring the distance beetween variables points, and how we can see in dataset, the attributes have differents scales, therefore, some variables like 'duration_ms' can influence more than others. Thus, it's important that we normalize all those variables we will consider to the classifier.

In [111]:
#Instance Standar Scaler tool to normalize dataset
scaler = StandardScaler()

#Separate varibles from targets
variables_columns = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness','speechiness', 'tempo','valence']
X, y = attributes_spotify[variables_columns], attributes_spotify['target']

#Fit standard scaler
scaler.fit(X)
#Normalizing variables
X_new = scaler.transform(X)
X_norm = pd.DataFrame(X_new, columns=X.columns)
X_norm

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,-0.717823,-0.946784,-2.802758,1.243563,-0.488057,1.718989,0.941467,3.251456,-0.305344,0.409433
1,0.628682,-0.356682,-1.113525,-1.677175,-0.374177,-0.481576,-1.487569,-0.716993,-0.621248,-0.808528
2,-0.721282,-1.046170,0.252482,1.448110,0.431039,-0.812948,1.250182,0.648830,1.438313,-1.314327
3,1.552026,0.034649,0.064918,0.201801,-0.488028,-0.578092,-0.007279,0.092713,-1.634068,0.498454
4,-0.647457,-1.654907,0.022289,0.596624,-0.488034,-0.608334,-0.184903,0.893521,-1.155639,-1.824576
...,...,...,...,...,...,...,...,...,...,...
2012,-0.719008,1.463318,0.035954,0.178017,0.639758,1.525957,-0.598119,-0.302130,0.241603,-0.104458
2013,0.009272,0.525365,-0.222863,-1.282352,-0.488037,0.194036,-0.728944,-0.710320,1.147560,1.287499
2014,-0.717169,0.394922,0.594350,1.291132,1.789544,0.419240,0.854251,-0.503444,0.127169,-0.699276
2015,2.963972,-1.356750,1.443615,-2.557202,2.726948,-0.462273,-2.242737,-0.619117,-0.047763,-1.387161


### Now that we have all normalized variables, let's train KNN classifier again.

In [112]:
#Instance KNN Classifier
knn = neighbors.KNeighborsClassifier()

#Create a hyperparameters dictionary for GridSeach using CrossValidation
params_grid = {'n_neighbors': [k for k in range(1, 100)]}

#Instance GridSearchCV to tesl all k's using 10 cross validations split.
knn_grid = GridSearchCV(knn, params_grid, cv=10)

#Fit model
knn_grid.fit(X_norm,y)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30, ...]})

In [113]:
#Find the best performance
knn_grid.best_params_

{'n_neighbors': 9}

In [114]:
#Find the best score performance
knn_grid.best_score_

0.7288015368701048

### Using the normalized variables we got 72.8% of accuracy, which is much better than the last results. So, it's very important to analyze if the attribute's scale are similar, because some variables can be way more influent than the others.